In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

I used the yfinance library to download historical stock data for TATAMOTORS.NS and NIFTY50:
TATAMOTORS.NS: The ticker symbol for TATA MOTORS on the NSE.
^NSEI: The ticker symbol for the NIFTY50 index.

In [ ]:
# Fetch data for NIFTY50 (symbol '^NSEI') and TATA MOTORS (symbol 'TATAMOTORS.NS')
nifty50 = yf.download('^NSEI', start='2023-04-01', end='2024-04-01')
tata_motors = yf.download('TATAMOTORS.NS', start='2023-04-01', end='2024-04-01')
# Display the first few rows of data
print("NIFTY50 Data:")
print(nifty50.head())

print("\nTATA MOTORS Data:")
print(tata_motors.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

NIFTY50 Data:
Price              Close          High           Low          Open  Volume
Ticker             ^NSEI         ^NSEI         ^NSEI         ^NSEI   ^NSEI
Date                                                                      
2023-04-03  17398.050781  17428.050781  17312.750000  17427.949219  230200
2023-04-05  17557.050781  17570.550781  17402.699219  17422.300781  248300
2023-04-06  17599.150391  17638.699219  17502.849609  17533.849609  242700
2023-04-10  17624.050781  17694.099609  17597.949219  17634.900391  254800
2023-04-11  17722.300781  17748.750000  17655.150391  17704.800781  304300

TATA MOTORS Data:
Price              Close          High           Low          Open  \
Ticker     TATAMOTORS.NS TATAMOTORS.NS TATAMOTORS.NS TATAMOTORS.NS   
Date                                                                 
2023-04-03    421.630737    422.326428    416.910063    420.388455   
2023-04-05    423.916504    426.102933    420.537501    421.630716   
2023-04-06    434

To calculate daily returns, I used the percentage change in the Close prices of both TATAMOTORS.NS and NIFTY50:

Daily returns are computed as the percentage change in the closing price from one day to the next:
Daily Return=(𝑃𝑡−𝑃𝑡−1)/𝑃𝑡−1

This is done using the pct_change() method from pandas.

In [ ]:
# Use Adjusted Close prices for calculations
tata_motors['Daily Return'] = tata_motors['Close'].pct_change()
nifty50['Daily Return'] = nifty50['Close'].pct_change()


To perform the regression analysis, I merged the daily returns of TATAMOTORS.NS and NIFTY50 into a single DataFrame.
The first row will have NaN values due to the pct_change() function, so I dropped that.

In [ ]:
data = pd.concat([tata_motors['Daily Return'], nifty50['Daily Return']], axis=1).dropna()
data.columns = ['TATA_MOTORS', 'NIFTY50']

To calculate alpha and beta, I used Ordinary Least Squares (OLS) regression:

Beta represents the sensitivity of TATA MOTORS returns to the market (NIFTY50) returns. It is the slope of the regression line.

Alpha represents the intercept, which is the excess return of TATA MOTORS over the market.

For regression, we need:

The dependent variable (y): This is the return of TATAMOTORS.NS.

The independent variable (X): This is the return of the NIFTY50.

In [ ]:
from sklearn.linear_model import LinearRegression
# Reshape the data for fitting in sklearn (make sure it's a 2D array for X)
X = data['NIFTY50'].values.reshape(-1, 1)  # Independent variable (Market return)
y = data['TATA_MOTORS'].values  # Dependent variable (TATA MOTORS return)

# Initialize and fit the Linear Regression model
model = LinearRegression()
model.fit(X, y)

# Getting the alpha (intercept) and beta (slope)
alpha = model.intercept_
beta = model.coef_[0]

In [ ]:
# Display results
print(f"Alpha (Intercept): {alpha}")
print(f"Beta (Slope): {beta}")

Alpha (Intercept): 0.0024941822222481888
Beta (Slope): 1.0940495680506135


Calculate the daily returns for both TATAMOTORS.NS and NIFTY50.
Compute the excess returns: Subtract the risk-free rate (7.015% annually) from the daily returns.
Calculate the Sharpe ratio using the formula:
Sharpe Ratio=(Mean of Excess Returns)/(Standard Deviation of Excess Returns)

Annualize the Sharpe ratio by multiplying by 252(assuming 252 trading days in a year).

In [ ]:
# Function to calculate Sharpe ratio (annualized)
risk_free_rate_annual = 7.015 / 100
risk_free_rate_daily = (1 + risk_free_rate_annual) ** (1/252) - 1

def calculate_sharpe_ratio(returns, risk_free_rate=risk_free_rate_daily):
    # Excess returns (subtracting the risk-free rate)
    excess_returns = returns - risk_free_rate
    # Sharpe ratio formula: mean of excess returns / standard deviation of excess returns
    sharpe_ratio_daily = excess_returns.mean() / excess_returns.std()
    # Annualize Sharpe ratio by multiplying by sqrt(252)
    sharpe_ratio_annual = sharpe_ratio_daily * np.sqrt(252)
    return sharpe_ratio_annual
# Calculate Sharpe ratios for TATAMOTORS and NIFTY50
sharpe_ratio_tata_motors = calculate_sharpe_ratio(tata_motors['Daily Return'])
sharpe_ratio_nifty50 = calculate_sharpe_ratio(nifty50['Daily Return'])

# Display results
print(f"Annualized Sharpe Ratio for TATAMOTORS: {sharpe_ratio_tata_motors}")
print(f"Annualized Sharpe Ratio for NIFTY50: {sharpe_ratio_nifty50}")



Annualized Sharpe Ratio for TATAMOTORS: 3.5684329777217294
Annualized Sharpe Ratio for NIFTY50: 1.988041196309432


Calculating Max Drawdown:
Calculate the cumulative returns for both shares (this represents the growth of an initial investment over time).

Track the running maximum of the cumulative returns up to each point.

Calculate the drawdown at each point, which is the difference between the current cumulative return and the running maximum.

The maximum drawdown is the largest negative value of the drawdowns, indicating the deepest peak-to-trough loss.

In [ ]:
# Cumulative Returns Calculation
tata_motors['Cumulative Return'] = (1 + tata_motors['Daily Return']).cumprod()
nifty50['Cumulative Return'] = (1 + nifty50['Daily Return']).cumprod()

# Running Maximum Calculation
tata_motors['Running Max'] = tata_motors['Cumulative Return'].cummax()
nifty50['Running Max'] = nifty50['Cumulative Return'].cummax()

# Drawdowns Calculation
tata_motors['Drawdown'] = (tata_motors['Cumulative Return'] - tata_motors['Running Max']) / tata_motors['Running Max']
nifty50['Drawdown'] = (nifty50['Cumulative Return'] - nifty50['Running Max']) / nifty50['Running Max']

# Maximum Drawdown Calculation
tata_motors_max_drawdown = tata_motors['Drawdown'].min()
nifty50_max_drawdown = nifty50['Drawdown'].min()

# Display the Maximum Drawdowns
print(f"Maximum Drawdown for TATAMOTORS.NS: {tata_motors_max_drawdown:.4f}")
print(f"Maximum Drawdown for NIFTY50: {nifty50_max_drawdown:.4f}")

Maximum Drawdown for TATAMOTORS.NS: -0.0951
Maximum Drawdown for NIFTY50: -0.0661
